In [63]:
import numpy as np
from bending_equations import Kt_key, Kts_key, Kf_key, Kfs_key

### Key Width

In [64]:
# Nominal key width
dg1 = 1.84 # diameter of shaft at gear 1
dp2 = 2.2 # diameter of shaft at pinion 2
dg2 = 2.51 # diameter of shaft at gear 2
dp1 = 1.47 # diameter of shaft at pinion 1

w = 0 

def key_width(d):
    if 0.312 < d <= 0.437:
        w = 0.093
    elif 0.437 < d <= 0.562:
        w = 0.125
    elif 0.562 < d <= 0.875:
        w = 0.187
    elif 0.875  < d <= 1.25:
        w = 0.25
    elif 1.25 < d <= 1.375:
        w = 0.312
    elif 1.375 < d <= 1.75:
        w = 0.375
    elif 1.75  < d <= 2.25:
        w = 0.5
    elif 2.25  < d <= 2.75:
        w = 0.625
    elif 2.75  < d <= 3.25:
        w = 0.75
    elif 3.25 < d <= 3.75:
        w = 0.875
    elif 3.75 < d <= 4.5:
        w = 1.0
    elif 4.5 < d <= 5.5:
        w = 1.25
    elif 5.5 < d <= 6.5:
        w = 1.5

    return w

In [65]:
wg1 = key_width(dg1) 
print('wg1 is', wg1)

wp2 = key_width(dp2) 
print('wp2 is', wp2)

wg2 = key_width(dg2)
print('wg2 is', wg2)

wp1 = key_width(dp1)
print('wp1 is', wp1)

wg1 is 0.5
wp2 is 0.5
wg2 is 0.625
wp1 is 0.375


### Forces & Torques

In [66]:
Se = 32500
Sut = 65000
Sy = 38000
Sf = Se
r = 0.010 # end mill radius

Ma_g1, Mm_g1, Ta_g1, Tm_g1 = 232.38, 985.58, 1307.25, 5664.76 
Ma_p2, Mm_p2, Ta_p2, Tm_p2 = 1745.3, 7505.82, 1307.25, 5664.76 
Ma_g2 , Mm_g2, Ta_g2, Tm_g2 = 0, 352.97, 10016.3909, 43404.36059
Ma_p1, Mm_p1, Ta_p1, Tm_p1 = 0, 2.2, 682.4464488, 2957.267945

Fa_g1, Fm_g1 = Ta_g1 / (dg1/2), Tm_g1 / (dg1/2)
Fa_p2, Fm_p2 = Ta_p2 / (dp2/2), Tm_p2 / (dp2/2)
Fa_g2, Fm_g2 = 10016.39 / (dg2/2), 43404.36 / (dg2/2)
Fa_p1, Fm_p1 = 682.4464 / (dp1/2), 2957.286 / (dp1/2)

In [67]:
def Safety_Factors(d, r, Fa, Fm, L, w, Ma, Mm, Ta, Tm):
    # Kt = Kt_key(d, r)
    Kf = Kf_key(d, r)
    Kfs = Kfs_key(d, r)
    A_shear = w * L
    tau_a = Fa / A_shear
    tau_m = Fm / A_shear

    # sigma_a = np.sqrt(3 * tau_a**2)
    # sigma_m = np.sqrt(3 * tau_m**2)

    # Nf = 1 / ((sigma_a/Se) + (sigma_m/Sut)) # fatigue SF

    A_bearing = w/2 * L
    sigma_max = (Fa + Fm) / A_bearing
    Ns = Sy / sigma_max # bearing SF for key


    # Compute new SF:
    Nf_new = (d ** 3 * np.pi / 32) / (((np.sqrt((Kf*Ma)**2 + (3/4)*(Kfs*Ta)**2))/Sf + np.sqrt((Kf*Mm)**2 + (3/4)*(Kfs*Tm)**2)/Sut))
    
    return Ns, Nf_new

In [68]:
# Bearing SF for key failure = Ns 
# Updated SF for shaft failure = Nf_new

# Want Ns < Nf_new for acceptable design ??

#### L of key for gear 1

In [69]:
Lg1 = 1.50 # max is 1.609
gear1 = Safety_Factors(dg1, r, Fa_g1, Fm_g1, Lg1, wg1, Ma_g1, Mm_g1, Ta_g1, Tm_g1)
Ns, Nf_new = gear1[0], gear1[1]
print('Ns gear 1 is', Ns, '\n''Nf_new gear 1 is', Nf_new)

if Ns < Nf_new:
    print('Acceptable design for gear 1 since Ns < Nf_New, Lg1 =', Lg1)
else:
    print('Unacceptable design since Ns > Nf_new')

Ns gear 1 is 1.8803759604475614 
Nf_new gear 1 is 2.017545135156302
Acceptable design for gear 1 since Ns < Nf_New, Lg1 = 1.5


#### L for key for pinion 2

In [70]:
Lp2 = 1.10 # max is 1.352
pinion2 = Safety_Factors(dp2, r,  Fa_p2, Fm_p2, Lp2, wp2, Ma_p2, Mm_p2, Ta_p2, Tm_p2)
Ns, Nf_new = pinion2[0], pinion2[1]
print('Ns pinion 2 is', Ns, '\n''Nf_new pinion 2 is', Nf_new)

if Ns < Nf_new:
    print('Acceptable design for pinion 2 since Ns < Nf_New, Lp2 =', Lp2)
else:
    print('Unacceptable design since Ns > Nf_new')

Ns pinion 2 is 1.6487354435808326 
Nf_new pinion 2 is 2.0269745577781064
Acceptable design for pinion 2 since Ns < Nf_New, Lp2 = 1.1


#### L of key for gear 2

In [71]:
Lg2 = 2.25 # max is 2.43
gear2 = Safety_Factors(dg2, r, Fa_g2, Fm_g2, Lg2, wg2, Ma_g2, Mm_g2, Ta_g2, Tm_g2)
Ns, Nf_new = gear2[0], gear2[1]
print('Ns gear 2 is', Ns, '\n''Nf_new gear 2 is', Nf_new)

if Ns < Nf_new:
    print('Acceptable design for gear 2 since Ns < Nf_New, Lg2 =', Lg2)
else:
    print('Unacceptable design since Ns > Nf_new')

Ns gear 2 is 0.6276967517303669 
Nf_new gear 2 is 0.6793443876057117
Acceptable design for gear 2 since Ns < Nf_New, Lg2 = 2.25


#### L for key for pinion 1

In [72]:
Lp1 = 1.20 # max is 1.39
pinion1 = Safety_Factors(dp1, r,  Fa_p1, Fm_p1, Lp1, wp1, Ma_p1, Mm_p1, Ta_p1, Tm_p1)
Ns, Nf_new = pinion1[0], pinion1[1]
print('Ns pinion 1 is', Ns, '\n''Nf_new pinion 1 is', Nf_new)

if Ns < Nf_new:
    print('Acceptable design for pinion 1 since Ns < Nf_New, Lp1 =', Lp1)
else:
    print('Unacceptable design since Ns > Nf_new')

Ns pinion 1 is 1.7265692389913059 
Nf_new pinion 1 is 2.0029703819665277
Acceptable design for pinion 1 since Ns < Nf_New, Lp1 = 1.2
